In [12]:
import sys
from PyQt5 import QtCore, QtWidgets, QtWebEngineWidgets
from lxml import html as htmlRenderer
import requests
import json
from datetime import date
import datetime
from random import *
import base64

In [2]:
def render(source_url):
    """Fully render HTML, JavaScript and all."""

    import sys
    from PyQt5.QtWidgets import QApplication
    from PyQt5.QtCore import QUrl
    from PyQt5.QtWebEngineWidgets import QWebEngineView

    class Render(QWebEngineView):
        def __init__(self, url):
            self.html = None
            self.app = QApplication(sys.argv)
            QWebEngineView.__init__(self)
            self.loadFinished.connect(self._loadFinished)
            #self.setHtml(html)
            self.load(QUrl(url))
            self.app.exec_()

        def _loadFinished(self, result):
            # This is an async call, you need to wait for this
            # to be called before closing the app
            self.page().toHtml(self._callable)

        def _callable(self, data):
            self.html = data
            # Data has been stored, it's safe to quit the app
            self.app.quit()

    return Render(source_url).html

In [3]:
url="https://www.lavanguardia.com/internacional/20190828/4711413454/bercow-ataque-johnon-parlamento-britanico.html"
renderUrl = render(url)
renderedPage = htmlRenderer.fromstring(renderUrl)

In [4]:
# i class="fa fa-comment"></i> <label class="livefyre-commentcount" data-lf-site-id="351112"
# 		data-lf-article-id="45a68962-c98c-11e9-a000-a9c287b54588" data-counter="total">0</label></a>

auxLinks = renderedPage.xpath("//label[@class='livefyre-commentcount']")

In [6]:
commentEl = auxLinks[0]

In [7]:

commentElVal = commentEl.get("data-lf-site-id")
commentElArticleId = commentEl.get("data-lf-article-id")

print(commentElVal)
print(commentElArticleId)

351112
45a68962-c98c-11e9-a000-a9c287b54588


# get total of comments

In [18]:
# Get Comments Info
urlParam = "{}:{}".format(commentElVal, commentElArticleId)
urlParamBase64 = str(base64.b64encode(urlParam.encode("utf-8")), "utf-8")
commentElArticleIdEncoded = str(base64.b64encode(commentElArticleId.encode("utf-8")), "utf-8")
urlInfoComments = "https://grupogodo1.bootstrap.fyre.co/api/v1.1/public/comments/ncomments/{}.json".format(urlParamBase64)


In [19]:
urlParamBase64

'MzUxMTEyOjQ1YTY4OTYyLWM5OGMtMTFlOS1hMDAwLWE5YzI4N2I1NDU4OA=='

In [20]:
commentElArticleIdEncoded

'NDVhNjg5NjItYzk4Yy0xMWU5LWEwMDAtYTljMjg3YjU0NTg4'

In [14]:
responseInfoComments = requests.get(urlInfoComments)

In [15]:
responseInfoComments

<Response [200]>

In [16]:
infoComments = json.loads(responseInfoComments.text)

In [17]:
infoComments

{'code': 200,
 'data': {'351112': {'45a68962-c98c-11e9-a000-a9c287b54588': {'facebook': 0,
    'feed': 0,
    'instagram': 0,
    'livefyre': 39,
    'total': 39,
    'twitter': 0}}},
 'status': 'ok'}

In [36]:
# https://data.livefyre.com/bs3/v3.1/grupogodo1.fyre.co/351112/ZGIwMDM2MjAtYzgxMC0xMWU5LWEwMDAtYTljMjg3YjU0NTg4/init
urlGetComments = "https://data.livefyre.com/bs3/v3.1/grupogodo1.fyre.co/{}/{}/init".format(commentElVal, commentElArticleIdEncoded)

In [37]:
urlGetComments

'https://data.livefyre.com/bs3/v3.1/grupogodo1.fyre.co/351112/NDVhNjg5NjItYzk4Yy0xMWU5LWEwMDAtYTljMjg3YjU0NTg4/init'

In [38]:
responseComments = requests.get(urlGetComments)

In [39]:
commentsResponse = json.loads(responseComments.text)

In [40]:
commentsResponse

{'collectionSettings': {'allowEditComments': False,
  'allowGuestComments': False,
  'archiveInfo': {'nPages': 2,
   'pageInfo': {'0': {'first': 1566996073,
     'last': 1567004685,
     'url': '/grupogodo1.fyre.co/351112/NDVhNjg5NjItYzk4Yy0xMWU5LWEwMDAtYTljMjg3YjU0NTg4/0.json'},
    '1': {'first': 1567006447,
     'last': 1567015495,
     'url': '/grupogodo1.fyre.co/351112/NDVhNjg5NjItYzk4Yy0xMWU5LWEwMDAtYTljMjg3YjU0NTg4/1.json'}},
   'pathBase': '/grupogodo1.fyre.co/351112/NDVhNjg5NjItYzk4Yy0xMWU5LWEwMDAtYTljMjg3YjU0NTg4/'},
  'bootstrapUrl': '/grupogodo1.fyre.co/351112/NDVhNjg5NjItYzk4Yy0xMWU5LWEwMDAtYTljMjg3YjU0NTg4/head.json',
  'checksum': '57fab8baeb7d64c22340a798a668cf87',
  'collectionId': '245002541',
  'commentsDisabled': False,
  'config': {'apps': {'analytics': {'livefyre': {'endpoint': '//beacon.livefyre.com/data'}}}},
  'editCommentInterval': 0,
  'event': 1567017908842672,
  'followers': 3,
  'nestLevel': 0,
  'networkId': 'grupogodo1.fyre.co',
  'numVisible': 39,
  'si

In [45]:

def extractComments(commentsObjectList, authorObjectList, urlNoticia="", specialCase = False):
    print(" \t -> parsing comments list with -{}- elements:".format(len(commentsObjectList)))
    parsedComments = []
    listToParse = commentsObjectList[1] if specialCase else commentsObjectList
    for commentObj in listToParse:
        contentObj = commentObj.get("content", None)
        isValidContent = "bodyHtml" in contentObj.keys()
        if isValidContent:
            fechaObj = datetime.datetime.fromtimestamp(contentObj["createdAt"])
            fechaStr = fechaObj.strftime("%d/%m/%Y-%H:%M:%S")
            fechaArr = fechaStr.split("-")
            parsedComment = {
                "urlNoticia": urlNoticia,
                "fecha": fechaArr[0],
                "hora": fechaArr[1],
                "user": authorObjectList[contentObj['authorId']]['displayName'],
                "commentario": contentObj['bodyHtml']
            }
            parsedComments.append(parsedComment)
    return parsedComments

In [46]:
parsedComments = extractComments(commentsResponse["headDocument"]['content'], commentsResponse["headDocument"]['authors'], url)

 	 -> parsing comments list with -55- elements:
 	-> is valid False
 	-> is valid False
 	-> is valid True
 	-> is valid False
 	-> is valid False
 	-> is valid False
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid False
 	-> is valid False
 	-> is valid True
 	-> is valid True
 	-> is valid False
 	-> is valid False
 	-> is valid False
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid False
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid False
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid False
 	-> is valid False
 	-> is valid False
 	-> is valid False
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> is valid True
 	-> 

In [49]:
print(parsedComments)

[{'urlNoticia': 'https://www.lavanguardia.com/internacional/20190828/4711413454/bercow-ataque-johnon-parlamento-britanico.html', 'fecha': '28/08/2019', 'hora': '17:34:07', 'user': 'Hispaniol', 'commentario': '<p>Después de perder un referéndum, después de perder unas elecciones, después de perder unas elecciones europeas. Despues de estar dos años sin ponerse de acuerdo sobre cómo salir después de todo eso los supuestos demócratas se escandalizan. Pero esto no es lo que la mayoría de ahí quiere? Pues puerta!</p>'}, {'urlNoticia': 'https://www.lavanguardia.com/internacional/20190828/4711413454/bercow-ataque-johnon-parlamento-britanico.html', 'fecha': '28/08/2019', 'hora': '16:47:48', 'user': 'Ull de Poll.....', 'commentario': '<p>Veis como no solo España se quiere petar la democracia...</p>'}, {'urlNoticia': 'https://www.lavanguardia.com/internacional/20190828/4711413454/bercow-ataque-johnon-parlamento-britanico.html', 'fecha': '28/08/2019', 'hora': '15:55:35', 'user': 'el señorito y vi

In [48]:
len(parsedComments)

39

In [50]:

today = date.today()
rootPath = "/home/cflores/cflores_workspace/comments-retriever/results"
fileName = "{}/{}-{}_{}_{}.json".format(rootPath, "lavanguardia", today.day, today.month, today.year)
with open(fileName, "w") as file:
    json.dump(parsedComments, file)
